In [1]:
# This cell should be exactly the same between all models

import pickle as pk

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from common import *
from util import *

# Load everything
scaled_data = {}
with np.load(PREPROCESSED_DATASET_FILEPATH) as npz_loader:
    for key in npz_loader:
        scaled_data[key] = npz_loader[key]
scaler = pk.load(open(PREPROCESSING_SCALER_FILEPATH, "rb"))

# Input and output dims
input_shape = tuple(list(scaled_data['x_train'].shape)[1:])
output_shape = tuple(list(scaled_data['y_train'].shape)[1:])
input_dims = np.product(input_shape)
output_dims = np.product(output_shape)

In [2]:
print("tensorflow: {}".format(tf.__version__))
print("numpy: {}".format(np.__version__))

tensorflow: 2.0.0
numpy: 1.17.0


In [3]:
# CONSTANTS:
MODEL_NAME = "LinearRegression"

In [4]:
# Construct TF model here

# Linear Regression
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=output_dims,activation='linear'),
        tf.keras.layers.Reshape(output_shape)
    ]
)

In [5]:
# hyperparameters and callbacks

BATCH_SIZE = 64
MAX_EPOCHS = 20

def scheduler(epoch, lr):
    if epoch<50:
        return lr
    elif epoch%20==0:
        return lr/10
    else:
        return lr
    
callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)
callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.compile(loss='mean_absolute_error',
              optimizer=tf.keras.optimizers.Adam(1e-3))

In [6]:
# Compile and fit model here
history = model.fit(
    x=scaled_data['x_train'],
    y=scaled_data['y_train'],
    batch_size=BATCH_SIZE,
    epochs=MAX_EPOCHS,
    validation_data=(scaled_data['x_val'],
    scaled_data['y_val']),
    callbacks=[callback_lr,callback_es]
)

Train on 23974 samples, validate on 3425 samples
Epoch 1/20
23974/23974 [==============================] - 76s 3ms/sample - loss: 0.3934 - val_loss: 0.3915
Epoch 2/20
23974/23974 [==============================] - 73s 3ms/sample - loss: 0.3395 - val_loss: 0.3550
Epoch 3/20
23974/23974 [==============================] - 71s 3ms/sample - loss: 0.3293 - val_loss: 0.3639
Epoch 4/20
23974/23974 [==============================] - 71s 3ms/sample - loss: 0.3276 - val_loss: 0.3418
Epoch 5/20
23974/23974 [==============================] - 71s 3ms/sample - loss: 0.3229 - val_loss: 0.3296
Epoch 6/20
23974/23974 [==============================] - 73s 3ms/sample - loss: 0.3218 - val_loss: 0.3271
Epoch 7/20
23974/23974 [==============================] - 75s 3ms/sample - loss: 0.3162 - val_loss: 0.3231
Epoch 8/20
23974/23974 [==============================] - 74s 3ms/sample - loss: 0.3159 - val_loss: 0.3239
Epoch 9/20
23974/23974 [==============================] - 76s 3ms/sample - loss: 0.3165 - val_l

In [7]:
# Compute unnormalized prediction loss

preds = {}

for split in ['test','val']:
    preds[split] = model.predict(scaled_data['x_'+split])
    preds[split] = scaler.inverse_transform(preds[split])

In [8]:
# Save model
model.save(model_name_to_model_filepath(MODEL_NAME))

# Save run info
run_info = {}
run_info["history"] = history.history
run_info["predictions"] = preds # idk whether this part makes sense for RNNs or not
pk.dump(run_info, open(model_name_to_run_info_filepath(MODEL_NAME), "wb"))